<a href="https://colab.research.google.com/github/dharshriya15/Thyroid_data_with_logistic_regression/blob/main/Exercise_Creating%2C_Reading_and_Writing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pitchfork-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F655%2F1252%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T032531Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D328ad7cd6e35a6efd29fbd7bc272b9161144eed2d405fe0047f7bd663073d5b304f49114d03bc5d5d38b8f8a440014a4ff65ca4b1012716edd2d046dfd6b8b39d371e46d71ceff0c87f6aa4513374d4f098678dde53882a23295724beb28ee7af533ad8fcd2054a8c7c6a2723003788328ad875862b49fab0699d2b170d6ddf0e0effe353c007405a04e58e360b3c59e7caabd082b6ba20717f434772280206ecd682b76d9ddc985ea9287cfb940ba0752789af89abb5d9fd99d60239983c76a608339173ab76ad3d00ab78445c17a91eedd72d8369fa7eb232dec645a3f11e7ce71876b60057f9a066e5d2547ddd121059f9f25bc5f2a0e80ff261e225113bc,chess:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2321%2F3919%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T032531Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D86b2bc9ec6b68fa48e27c4210342821d04c9c8286210ae51e993286e1db9c94c6eeddb69425d54e1ffd26b8fef7e075d71d59a810aa18138b643577cb2f10a3773f4eba43373a52a4734cdd820e110a903200eb63e09aad0a25cb6c4be4c092fbb814259903d745286686150deb2df21cb883803a75f6a2ef257c54ac4d4e47bd694d2e22616e4b436f6268dc734fdadf487412eef6ebcc6a7e52acf3601d41e8707ba89b4dd137a1d811be10c67dea4eb4ffd19603abe5c6d02f5b7a8b39ce0479f3210ca660544425eab36ba6c4dd98dce80d9bd5c1d8ed78f4cfe4ceaf2e4636e36be9d719faf5ddeea60648d7b923bc64582f1b8e6bb71462029d5a2651f,kepler-exoplanet-search-results:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2894%2F4877%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T032531Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Daf03e2324fbfe35879461d352d8b5c1707ccffa4c8377b64b9af4d12816af7294e5bfd32d3fdd7c5bdf9dd7e43bfa156970307cf4e0ff712a6f035ac684c3f8ed292dedb3249b855d2fde8bbc96ffebd0d96933844132e6001109b36d461c558b1c0207c557358a6bb8687d2737de1ea6fa97a8f2dc23b47dc9932ed4a2f0d6c707c28131a0ea99893f279cc9a4dc0409926ea4d06c08275bfbf4cfd852cedc050c557f1079c8316e809d6a51ba325bc7a75fccb475e431a913493fb5f3ef96e43574245d811b35830191ac7a33403d99df063f767e3ba1b450178a6ebf64b771e1b27260428a305e80f76c2af26dbbcaa3061962dc578b7badc167dbf5af47a,things-on-reddit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3491%2F5624%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T032531Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da4482b2163cf138b0e55bea435d38b0e065e7d9a27e127568c95551c2a5a56155ac6bcaeb50be9710993daff4566e06eef31ae193d9325e2674982d1c1a7dcca1a76003705a23fc818bdde80bfd62110df7d203d9e64bf3e0b0235ac53774c0ccbff60e99e1415c45e7760965afaf455c5801b6131240a240932675e4493692aaf0d1528e139dce33e89639069d0f455375de1a38984a67bbe61aa31701cd4f8e658bec13b7745a7b74abeef1c2301fba56969b24c529e71d7791ac4830a4883bdb9733c68701ba33c3f62271017c497d60f020012614134ad71da4c7dc8d50dd612de64152fcf6d0f023751b17b1404bb854d069ef770ac605c785f131c9e71,wine-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1442%2F8172%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T032532Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D56ef07fd2057e5c8461bd1d1649fab883da0ef72116bb27121d8c828d99a9af3dd636f2f99675bb4f0e8719a0fd504d3b4c7c350b9aff6c3bfe3c030e3412dbc05ffdc140dff196b3513876de881eedda03b3f3b92f42b3ccd9de05d8411e3ae2216727bbc7c80fe075b59dd39871e5cb5bd3a3012a2b37a7759514b0d629b774a77d828bf1c68c0e00430201d8dbe54ba8763ce9ffd917ae6ff4165397eafae1c09dbac3ca5215bf35c98786cf7b6f4f4717802c4ec07df2989fc1d727b6f5d4ca3fd67cddce54f39000d18c9242e24eea2bc19c13370c032fef0230ce0815f33c9a14744c8826da3a01a4ef42bfc9d4ab909dd75c3ee074d567e16337db211,ramen-ratings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9366%2F13206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T032532Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da33367ef911ca250bb4402a43a05a10d01d91870dede590843927a3b6e8279cf1c5de586abb590f02816ae453e4b83f5c95f7682ff050d5ad524d5a1b2d5a010fe6ca7d705029402c042cf21f8ffcc07019d13e669e8431b250633bdf15d16c17464ab8dd41d3ecf2043de78a1fa8648b3dc868be59cc149149184fcb5d65bc19c220a76e550bd4463306fc094a94d4e9400f87729271cdb6578b01b59a753be571dea434be139b11c895d756edff64252f32682afd8c243cf355428751b93e927ef5ab77712ad70dac6c28e4c476797ae3f0cc5ce23627e0c969eee2271ba48bfe47ae17a450c611c4fe7fa5c4dc512ae37e8bb5eb8591abf7e28caa9d95049,powerlifting-database:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F179555%2F403916%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T032532Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D98c94e059bbb8c770db0b3dbe4a30640b1d3dcdf1fe4ae55f40e75f1de019edca932a9a2bd2c2291fad687e1a216ef0b794977d511acf45790f750c99f4978510221dbcba743132ce9d951eba028e32faac0a6bb44c5c86f376afd5238ff8bd6482eeb93138ba2016d21ee26cadc0079109e7a65f035f3228b07642650b3e5492d6dddf7778f94548df85ce0daeab33fc02c4a7328e237978f14b8fc8f50b3e667090714c89d82813484e5a2a902e60da8f0a00b44bab5852598dfceabce64f72e3211a508e579e9e3cdd81b2c616f4999a5c6d74d57dcd569f89f0c44fe60e07b86b85dc9ac93fa0447e28a469d2a0923360ae8476fd9bd88909b83412d0b62,youtube-new:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4549%2F466349%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T032532Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D38e06d6b056a318a2173c70abed49d742e593fde88ad35972974f2916d061dcd8784041fcb0a7c2e28a0cdfba197a41a35bce2774eba1c1b89f25fe466620b0370f0358614177918098ac1fbb40e15cc768933141cddd67880ee741b6ce1c45ae5d1d5d5660dbeececfe3e32f95691139aaedb157c25c4b746f4a6530052a0371351dd089002a26a7266342a94aaaf9a3a6353b6dd89730c67312c9775a0d951b900d7de1a080629dc2144d9f4c8c7696c62a5052a0fc9e40fc64ded0f799354548d8322637a21f90539416974299c3c41e220ed54a6f9c5323b35098a63b788e57ef9002d054bf38797d562eeeffc6ecd30106b257e81bd4ea2c91485e4ae11,188-million-us-wildfires:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2478%2F1151655%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T032532Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da42f2d98e6672dc49d5bb47c71440052c84ba0b6c5f03c2cd327fc6102cac5c105931c0d253855e39da3e278118bf82260c60e8feeb761d426e73899ec77a822b9fb266dd7695a0de46efc8198d8c37a803fb870ae2d2766cffd07869b1802618b9418b696075650c915f470c84f2be6177ff235c270a34353f13c6f30e96da552494275854e3215287c2c036ad3bc41a816cb2a544840d8e3c8c9bc74a6729d78b02fa6cc14cb860c00716542eed506c170a48d6a05d1bbe300cf32f2488b9bd4c9623214dffa670456056c5f28023e46c0233214aa26ce78191d27c50d827f05021a09f43177d28bf6406df41ea4dbcc483c8ff82ec7d19ca1104e8fac7598,publicassistance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F10128%2F5438389%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T032532Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5f7fc78be4fc99d0362c2d4dbf1ae22d9d3dee67911ac2237952b8e185cc287470cd8c3e9b04707df41503954b6fbd6ddb6b3a358480ed2344e16ef28f741552445f31b5eb90f53a1a007c48e7206b274a9e6165164012a0546ca87130629693cd2bab8a402f82dbef97f084ce924b73c06a4d30ec921ab0d5b95a9d3e70ffbec3d15ebf17ff859254b56e319299271fab6f4e6902d3f55dfdee1bf02fb6f880ab15a631a163eafee1c73b65e061e1cd45732d285e7b6c4925f08eb855441a76017bcff2dd580c8f6e902ed135cafc8e99e2bd379d87093e0500905642241172277ee25335a43e496bf46e8e0036c55d718137026bc4fe12e798808d398af3c6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Pandas](https://www.kaggle.com/learn/pandas) course.  You can reference the tutorial at [this link](https://www.kaggle.com/residentmario/creating-reading-and-writing).**

---


# Introduction

The first step in most data analytics projects is reading the data file. In this exercise, you'll create Series and DataFrame objects, both by hand and by reading data files.

Run the code cell below to load libraries you will need (including code to check your answers).

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 5)
from learntools.core import binder; binder.bind(globals())
from learntools.pandas.creating_reading_and_writing import *
print("Setup complete.")

# Exercises

## 1.

In the cell below, create a DataFrame `fruits` that looks like this:

![](https://storage.googleapis.com/kaggle-media/learn/images/Ax3pp2A.png)

In [ ]:
# Your code goes here. Create a dataframe matching the above diagram and assign it to the variable fruits.
import pandas as pd
fruits = pd.DataFrame({'Apples':[30],'Bananas':[21]})

# Check your answer
q1.check()
fruits

In [ ]:
#q1.hint()
#q1.solution()

## 2.

Create a dataframe `fruit_sales` that matches the diagram below:

![](https://storage.googleapis.com/kaggle-media/learn/images/CHPn7ZF.png)

In [ ]:
# Your code goes here. Create a dataframe matching the above diagram and assign it to the variable fruit_sales.
import pandas as pd
fruit_sales =  pd.DataFrame({'Apples':[35,41],'Bananas':[21,34]},index=["2017 Sales","2018 Sales"])
# Check your answer
q2.check()
fruit_sales

In [ ]:
#q2.hint()
#q2.solution()

## 3.

Create a variable `ingredients` with a Series that looks like:

```
Flour     4 cups
Milk       1 cup
Eggs     2 large
Spam       1 can
Name: Dinner, dtype: object
```

In [ ]:
ingredients =pd.Series(["4 cups","1 cup","2 large","1 can"],index=["Flour","Milk","Eggs","Spam"],name="Dinner",dtype="object")

# Check your answer
q3.check()
ingredients

In [ ]:
#q3.hint()
#q3.solution()

## 4.

Read the following csv dataset of wine reviews into a DataFrame called `reviews`:

![](https://storage.googleapis.com/kaggle-media/learn/images/74RCZtU.png)

The filepath to the csv file is `../input/wine-reviews/winemag-data_first150k.csv`. The first few lines look like:

```
,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,"This tremendous 100% varietal wine[...]",Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and[...]",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,,Tinta de Toro,Bodega Carmen Rodríguez
```

In [ ]:
import pandas as pd
reviews = pd.read_csv("../input/wine-reviews/winemag-data_first150k.csv",index_col=0)

# Check your answer
q4.check()
reviews

In [ ]:
#q4.hint()
#q4.solution()

## 5.

Run the cell below to create and display a DataFrame called `animals`:

In [ ]:
animals = pd.DataFrame({'Cows': [12, 20], 'Goats': [22, 19]}, index=['Year 1', 'Year 2'])
animals

In the cell below, write code to save this DataFrame to disk as a csv file with the name `cows_and_goats.csv`.

In [ ]:
# Your code goes here
animals.to_csv("cows_and_goats.csv")
# Check your answer
q5.check()

In [ ]:
#q5.hint()
#q5.solution()

# Keep going

Move on to learn about **[indexing, selecting and assigning](https://www.kaggle.com/residentmario/indexing-selecting-assigning)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/pandas/discussion) to chat with other learners.*